In [9]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

In [10]:

params = {
	"latitude": 21.02,
	"longitude": 105.85,
	"start_date": "2025-01-01",
	"end_date": "2025-05-15",
	"daily": ["relative_humidity_2m_mean", "temperature_2m_min", "temperature_2m_max", "rain_sum"],
	"timezone": "Asia/Bangkok"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_relative_humidity_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(2).ValuesAsNumpy()
daily_rain_sum = daily.Variables(3).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["relative_humidity_2m_mean"] = daily_relative_humidity_2m_mean
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["rain_sum"] = daily_rain_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates 21.054479598999023°N 105.8071060180664°E
Elevation 20.0 m asl
Timezone b'Asia/Bangkok'b'GMT+7'
Timezone difference to GMT+0 25200 s
                         date  relative_humidity_2m_mean  temperature_2m_min  \
0   2024-12-31 17:00:00+00:00                  70.369194           12.915500   
1   2025-01-01 17:00:00+00:00                  65.792099           13.465500   
2   2025-01-02 17:00:00+00:00                  60.062744           14.265500   
3   2025-01-03 17:00:00+00:00                  63.819340           13.965500   
4   2025-01-04 17:00:00+00:00                  59.340221           14.265500   
..                        ...                        ...                 ...   
130 2025-05-10 17:00:00+00:00                  75.902489           20.915501   
131 2025-05-11 17:00:00+00:00                  71.557739           19.565500   
132 2025-05-12 17:00:00+00:00                  77.699509           21.165501   
133 2025-05-13 17:00:00+00:00                  85.172951